In [2]:
import csv
import numpy as np
import pandas as pd
import os, re
import codecs
import spacy
nlp = spacy.load('en')
import gensim
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary
import pickle

In [3]:
papers = pd.read_csv('/Users/macbook/Master--Y2/IR/nips-papers/papers.csv')

#author with unique id
authors = pd.read_csv('/Users/macbook/Master--Y2/IR/nips-papers/authors.csv')
#paper id + author id
paper_author = pd.read_csv('/Users/macbook/Master--Y2/IR/nips-papers/paper_authors.csv')

#papers = papers.sort['year']
paper_text = papers['paper_text']
paper_title = papers['title']
del paper_author['id']



In [ ]:
#Create author2doc
author2doc = dict()
for row_index,row in authors.iterrows():
    ids =[]
    author_id = row['id']
    author_name = row['name']
    #author_name = re.sub('\s', '', author_name_1)
    for row_index1, row_1 in paper_author.iterrows():
        if row_1['author_id'] == author_id:
            paper_id = row_1['paper_id']
            ids.append(paper_id)
    if not author2doc.get(author_name):
        # This is a new author.
        author2doc[author_name] = []
     # Add document IDs to author.
    author2doc[author_name].extend([id for id in ids])

In [ ]:
# Store data (serialize)
with open('author2doc.pickle', 'wb') as handle:
    pickle.dump(author2doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
author2doc = pickle.load(open('author2doc.pickle', 'rb'))

In [5]:
author2doc_1={}
for k, v in author2doc.items():
    k_1 = re.sub('\s', '', k)
    k_1 = k_1.lower()
    author2doc_1[k_1] = v

In [5]:
# Use an integer ID in author2doc, instead of the IDs provided in the NIPS dataset.
# Mapping from ID of document in NIPS datast, to an integer ID.
doc_ids = list(set(papers['id']))
doc_id_dict = dict(zip(doc_ids, range(len(doc_ids))))
# Replace NIPS IDs by integer IDs.
for a, a_doc_ids in author2doc_1.items():
    for i, doc_id in enumerate(a_doc_ids):
        author2doc_1[a][i] = doc_id_dict[doc_id]


In [6]:
%%time
processed_docs = []    
#docs_1 = docs
STOPWORDS = ['(a)','(b)','(c)','(d)','(e)','(f)','(g)','(h)','(1)','(2)','(3)','(4)','(5)',\
             '(i)','(ii)','(iii)','(iiii)','(v)']
for doc in nlp.pipe(paper_title, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities.
    
    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords and remove single letter, e.g.,'x','y'.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop and len(token)>1]
    
     
    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in STOPWORDS]
    #doc = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', doc)
    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

docs = processed_docs




CPU times: user 2.72 s, sys: 17.7 ms, total: 2.73 s
Wall time: 2.75 s


In [7]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.     
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 20
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ =dictionary[1]  # This sort of "initializes" dictionary.id2token.

In [8]:
# Vectorize data.
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]



In [9]:
print('Number of authors: %d' % len(author2doc))
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of authors: 8595
Number of unique tokens: 411
Number of documents: 6560


In [ ]:
model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                     author2doc = author2doc_1,\
                    chunksize=200,\
                    passes=100, gamma_threshold=1e-10,\
                    eval_every=0, iterations=1, random_state=1)

In [11]:
%%time
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                     author2doc = author2doc_1,\
                    chunksize=100,\
                    passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 12min 34s, sys: 4.19 s, total: 12min 38s
Wall time: 12min 40s


In [ ]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)
# Save model.
model.save('/tmp/model.atmodel')
# Load model.

In [10]:
model = AuthorTopicModel.load('/tmp/model.atmodel')
model.show_topics(num_topics=10, num_words=10, log=False, formatted=True)

[(0,
  '0.086*"analysis" + 0.081*"data" + 0.052*"high" + 0.050*"matrix" + 0.045*"models" + 0.040*"order" + 0.039*"clustering" + 0.038*"distributed" + 0.036*"dimensional" + 0.036*"structured"'),
 (1,
  '0.184*"learn" + 0.106*"stochastic" + 0.070*"sparse" + 0.058*"gradient" + 0.038*"random" + 0.034*"descent" + 0.030*"temporal" + 0.024*"problem" + 0.023*"active" + 0.022*"search"'),
 (2,
  '0.093*"linear" + 0.080*"efficient" + 0.050*"learning" + 0.041*"control" + 0.034*"machine" + 0.031*"local" + 0.029*"matching" + 0.028*"regularization" + 0.025*"distribution" + 0.025*"classification"'),
 (3,
  '0.151*"neural" + 0.151*"network" + 0.068*"estimation" + 0.063*"non" + 0.050*"dynamic" + 0.048*"online" + 0.048*"networks" + 0.027*"latent" + 0.026*"submodular" + 0.021*"net"'),
 (4,
  '0.299*"learning" + 0.056*"base" + 0.046*"function" + 0.044*"reinforcement" + 0.042*"multi" + 0.027*"graph" + 0.027*"application" + 0.025*"spectral" + 0.020*"algorithm" + 0.020*"inference"'),
 (5,
  '0.100*"optimizati

In [11]:
from pprint import pprint

def show_author(name):
    print('\n%s' % name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    pprint([(topic[0], topic[1]) for topic in model[name]])

In [12]:
show_author('zhaoranwang')


zhaoranwang
Docs: [5572, 5209, 5690, 5877, 5970, 5871, 6439, 6463, 6051, 6475, 6263]
Topics:
[(1, 0.33019163573243099), (2, 0.22859663373031203), (9, 0.39167087869323086)]


In [13]:
##find document topics
from sklearn import preprocessing
def getdocumenttopics(corpus):
    all_doc_topic=[]
    all_doc_topic_prob=[]
    for i in range(len(corpus)):
        topicseq=[]
        res = corpus[i]
        lst=[]
        for j in range(len(res)):
            topic = res[j][0]            
            count = res[j][1]
            #get the possibility 
            if model.get_term_topics(topic):
                a_topic =  model.get_term_topics(topic)[0][0]
                pos = count* model.get_term_topics(topic)[0][1]
                lst.append(pos)               
                topicseq.append(a_topic) 
        #Scale to [0,1]  
        lst_scaled= [item/np.sum(lst) for item in lst]
        doc_topic_prob = list(zip(topicseq,lst_scaled))
        all_doc_topic.append(topicseq)
        all_doc_topic_prob.append(doc_topic_prob) 
    return all_doc_topic, all_doc_topic_prob

all_topic, all_doc_topic_prob =getdocumenttopics(corpus)


from collections import Counter
for i in range(len(all_doc_topic_prob)):
    c = Counter()
    for topic, prob in all_doc_topic_prob[i]:
        c.update({topic: prob}) 
    all_doc_topic_prob[i] = list(c.items())
for i in range(len(all_topic)):
    all_topic[i]=list(set(all_topic[i]))



In [14]:
def findyearseq(doc_ids):
    year = [] 
    for id in doc_ids:
        for j,paper_id in enumerate(papers['id']):
            if id==paper_id:
                year.append(year_seq[j])
    return year   

In [15]:
def topicseq(name):
    topics=[]
    for i,doc_id in enumerate(model.author2doc[name]):  
        # each document we choose one topic
        doc_topics = all_topic[doc_id]
        topics.append(doc_topics) 
    return topics

In [16]:
# trends of research direction
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
def trendofresearch(name):
    # past current research direction
    print(show_author(name))
    doc_ids = model.author2doc[name]
    topics = topicseq(name)
    #print(topics)
    length = len(topics)
    print('length:',length)
    flat_list = [item for sublist in topics for item in sublist]
    topic_dict = {x:flat_list.count(x) for x in set(flat_list)}
    print('topic_dict:',topic_dict)
    major_topic=[]
    for topic, count in topic_dict.items():
        if count/length >0.5:
            major_topic.append(topic)
    if len(major_topic)>1:
        print('This author works on variety topics: ', major_topic)
    if len(major_topic)==1:
        print('This author focus on one topic',major_topic)
    if len(major_topic)==0:
        print('This author changes topics frequentyly', major_topic)
    
    




In [17]:
trendofresearch('zhaoranwang')


zhaoranwang
Docs: [5572, 5209, 5690, 5877, 5970, 5871, 6439, 6463, 6051, 6475, 6263]
Topics:
[(1, 0.33019163573243099), (2, 0.22859663373031203), (9, 0.39167087869323086)]
None
length: 11
topic_dict: {0: 7, 1: 1, 2: 3, 3: 1, 4: 2, 5: 2, 6: 1, 7: 3, 8: 2, 9: 3}
This author focus on one topic [0]


In [18]:
## plot pie chart or radar chart to visualize the distribution of the topics for each author    

import pygal
def visualizetopicdistribution(name):
    radar_chart = pygal.Radar()
    radar_chart.title = 'Topic distribution For Author' + name
    res = model[name]
    topic_seq = ['topic' + ' ' + str(i) for i in range(10)]

    topic_prob = np.zeros(10)
    for i in range(len(res)):
        topic = res[i][0]
        prob = res[i][1]
        topic_prob[topic] = prob
    doc_ids = model.author2doc[name]
    topics = topicseq(name)
    length = len(topics)

    flat_list = [item for sublist in topics for item in sublist]
    topic_dict = {x:flat_list.count(x) for x in set(flat_list)}
    topic_prob_1 = np.zeros(10)
    for topic, value in topic_dict.items():
        topic_prob_1[topic]=value/sum(topic_dict.values())


    radar_chart.x_labels = topic_seq
    radar_chart.add('Topics proabablity by ATM', topic_prob)
    radar_chart.add('Topics proabablity by Frequency', topic_prob_1)
    radar_chart.render_to_file('radar_chart.svg')



In [19]:
visualizetopicdistribution('zhaoranwang')